In [1]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re
import time
from IPython.display import clear_output
import multiprocessing
import threading
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

# BETWAY - Futbol

In [2]:
''' NAVIGATOR '''
bet_way = 'https://betway.bet.ar/es-ar/sports/lve/soccer'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#bway = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bway = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bway.get(bet_way)
bway.maximize_window()

#upcoming = WebDriverWait(bway, 4).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@collectionitem, "UPCOMING")]')))
#upcoming.click()

In [4]:
dropdown = WebDriverWait(bway, 4).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[1]')))
dropdown.click()

total_goals = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div[1]/div[2]')
total_goals.click()


In [31]:
a = bway.find_element(By.XPATH, path)
a.click()
#bway.execute_script("arguments[0].click()", a)

In [19]:
a =bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[2]/div[1]/div/div[1]/div[2]/div/div[1]').text.split('\n')
for idx, c in enumerate(a):
    print(idx, c)

0 1
1 -
2 1
3 Boca Juniors -CA Aldosivi
4 1ª mitad
5 ,
6 35'
7 1,55
8 3,10
9 7,50


In [21]:
def betway():
    df_bway = pd.DataFrame(columns =['teams', '1_bway', 'X_bway', '2_bway'])
    df_bway2 = pd.DataFrame(columns=['teams', 'under_bway', 'goals_bway', 'over_bway'])
    for j in range(30):
        for i in range(5):
            try:
                elemento = bway.find_element(By.XPATH, f'/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[2]/div[1]/div/div[{j}]/div[2]/div/div[{i}]').text.split('\n')
                
                players_bway = elemento[3]
                home = elemento[7]
                tie  = elemento[8]
                away = elemento[9]

                df_bway.loc[len(df_bway.index)] = [players_bway, home, tie, away]
                print(players_bway, '|         |', f'j: {j}, i: {i} ')

            except:
                pass
    dropdown = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[1]')
    dropdown.click()
    time.sleep(1.5)
    TotalGoals = bway.find_element(By.CSS_SELECTOR, '[collectionitem="total-goals"]')
    TotalGoals.click()
    df_bway.to_csv('CSV\df_bway.csv')
    for j in range(20):
        for i in range(10):
            try:
                elemento = bway.find_element(By.XPATH, f'/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[2]/div[1]/div/div[{j}]/div[2]/div/div[{i}]').text.split('\n')
                teams = elemento[3]
                goals = elemento[7].replace(',', '.')
                under = elemento[8].replace(',', '.')
                over  = elemento[10].replace(',', '.')
                df_bway2.loc[len(df_bway2.index)] = [teams, under, goals, over]
                print(teams, f'||     goals: {goals}, under: {under}, over: {over} ')
            except:
                pass

    df_bway = pd.merge(df_bway, df_bway2, on='teams')
    dropdown.click()
    time.sleep(1.5)
    HomeXaway = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div[1]/div[1]')
    HomeXaway.click()
    df_bway.to_csv('CSV\df_bway_pre.csv')
    return df_bway



In [22]:
betway()

Boca Juniors -CA Aldosivi |         | j: 1, i: 1 
Atletico Mineiro -Ceara |         | j: 2, i: 1 
Coritiba -Bragantino |         | j: 2, i: 2 
Fluminense -América Mineiro |         | j: 2, i: 3 
Austin -Colorado Rapids |         | j: 3, i: 1 
Dallas -Sporting Kansas City |         | j: 3, i: 2 
Houston Dynamo -Los Angeles Galaxy |         | j: 3, i: 3 
Los Ángeles -Nashville |         | j: 3, i: 4 
Colo-Colo -Curicó Unido |         | j: 6, i: 1 
Boca Juniors -CA Aldosivi ||     goals: 3.5, under: 1.80, over: 1.90 
Atletico Mineiro -Ceara ||     goals: 1.5, under: 1.95, over: 1.75 
Coritiba -Bragantino ||     goals: 3.5, under: 1.90, over: 1.80 
Fluminense -América Mineiro ||     goals: 2.5, under: 3.5, over: 2.5 
Austin -Colorado Rapids ||     goals: 2.5, under: 1.95, over: 1.75 
Dallas -Sporting Kansas City ||     goals: 2.5, under: 2.40, over: 1.50 
Houston Dynamo -Los Angeles Galaxy ||     goals: 3.5, under: 1.72, over: 2.00 
Los Ángeles -Nashville ||     goals: 2.5, under: 1.85, ov

,teams,1_bway,X_bway,2_bway,under_bway,goals_bway,over_bway
0,Boca Juniors -CA Aldosivi,"1,55","3,10","7,50",1.80,3.5,1.90
1,Atletico Mineiro -Ceara,"1,61","2,87","7,50",1.95,1.5,1.75
2,Coritiba -Bragantino,"5,25","2,70","1,85",1.90,3.5,1.80
3,Fluminense -América Mineiro,"8,00","4,75","1,30",3.5,2.5,2.5
4,Austin -Colorado Rapids,"1,85","3,10","4,20",1.95,2.5,1.75
5,Dallas -Sporting Kansas City,"1,70","3,30","4,75",2.40,2.5,1.50
6,Houston Dynamo -Los Angeles Galaxy,"2,05","3,60","3,00",1.72,3.5,2.00
7,Los Ángeles -Nashville,"1,53","3,80","5,50",1.85,2.5,1.85
8,Colo-Colo -Curicó Unido,"1,47","3,30","8,50",1.72,3.5,2.00


# BWAY PREMATCH

In [4]:
''' NAVIGATOR '''
bet_way = 'https://betway.bet.ar/es-ar/sports/lve/soccer'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#bway = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bway = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bway.get(bet_way)
bway.maximize_window()

upcoming = WebDriverWait(bway, 4).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@collectionitem, "UPCOMING")]')))
upcoming.click()

C:\Users\KNL\AppData\Local\Temp\ipykernel_24284\2660830590.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  bway = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)


In [16]:
def betway_pre():
    df_bway = pd.DataFrame(columns=['teams', '1_bway', 'X_bway', '2_bway'])
    df_bway2 = pd.DataFrame(columns=['teams', 'under_bway', 'goals_bway', 'over_bway'])
    for j in range(20):
        for i in range(10):
            try:
                elemento = bway.find_element(By.XPATH, f'//div[{j}]/div/div/div[@class="oneLineEventItem"][{i}]').text.split('\n')
                teams = elemento[1]

                home = elemento[2]
                tie  = elemento[3]
                away = elemento[4]
                df_bway.loc[len(df_bway.index)] = [teams, home, tie, away]
            except:
                pass
    ''' Clicking to under/over marker '''
    dropdown = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[1]')
    TotalGoals = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div[1]/div[2]')
    dropdown.click()
    TotalGoals.click()

    for j in range(20):
        for i in range(10):
            try:
                elemento = bway.find_element(By.XPATH, f'//div[{j}]/div/div/div[@class="oneLineEventItem"][{i}]').text.split('\n')
                teams = elemento[1]
                goals = elemento[2]
                under = elemento[3]
                over  = elemento[5]
                df_bway2.loc[len(df_bway2.index)] = [teams, under, goals, over]
                print(teams, f'||     goals: {goals}, under: {under}, over: {over} ')
            except:
                pass
    df_bway = pd.merge(df_bway, df_bway2, on='teams')
    dropdown.click()
    HomeXaway = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div[1]/div[1]')
    HomeXaway.click()
    df_bway.to_csv('CSV\df_bway_pre.csv')
    return df_bway

In [10]:
betway_pre()

,teams,1_bway,X_bway,2_bway,under_bway,goals_bway,over_bway
0,Boyaca Patriotas -Union Magdalena Santa Marta,"1,70","3,30","5,00","2,40","2,5","1,50"
1,Godoy Cruz -San Lorenzo,"2,15","3,10","3,20","1,50","1,5","2,40"
2,Newells Old Boys -Arsenal de Sarandi,"2,05","3,00","3,60","1,50","1,5","2,40"
3,CA Sarmiento -Tigre,"3,00","3,10","2,25","2,25","2,5","1,57"
4,Bayern de Múnich -Viktoria Plzeň,"1,06","13,00","34,00","2,20","4,5","1,65"
5,O. Marsella -Sporting de Lisboa,"2,25","3,40","3,10","1,90","2,5","1,87"
6,Ajax -Nápoles,"2,25","3,75","2,90","1,60","2,5","2,30"
7,Club Brujas -Atlético de Madrid,"3,75","3,40","2,00","2,00","2,5","1,77"
8,Inter -FC Barcelona,"4,00","4,00","1,77","1,60","2,5","2,30"
9,Liverpool -Glasgow Rangers,"1,16","8,00","13,00","1,85","3,5","1,90"


In [ ]:
dia_colapsado = bway.find_elements(By.CSS_SELECTOR, '[collectionitem*="2022"]>div.collapsableHeader[collapsed="true"]')
for dia in dia_colapsado:
    try:
        dia.click()
        print('click')
        time.sleep(5)
    except:
        pass